In [36]:
from autogen import ConversableAgent
import os
config_list = [
    {
        # Let's choose the Mixtral 8x7B model
        "model": "meta-llama/Llama-3-70b-chat-hf",
        # Provide your Together.AI API key here or put it into the TOGETHER_API_KEY environment variable.
        "api_key": os.environ.get("TOGETHER_API_KEY"),
        # We specify the API Type as 'together' so it uses the Together.AI client class
        "api_type": "together",
        "temperature": 0.8,
        "top_p": 0.9,
        "repetition_penalty": 1.0,
        "stream": False,
    }
]

In [37]:
suspect = ConversableAgent(
    name="suspect",
    system_message=
    """You are an advanced conversational AI designed to simulate mistaken interactions. Your task is to engage in a conversation where you initially address the user as if they are someone else, with a random intention. After a few exchanges, you realize your mistake and adjust the conversation accordingly. Use the following guidelines:
1.Start the conversation with a friendly greeting, addressing the user by a common name.
2.Introduce a random intention or topic (e.g., discussing plans, asking about a project, sharing some news).
3.Realize your mistake and apologize for the mix-up.
4.Hang up the phone
Keep your response short and concise, same as in a real phone call. Do not add actions or emotional cues or asterisk.
You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

In [38]:
import pandas as pd
import random
import string

# Function to initiate a chat and store the result
def initiate_chat_and_store(initial_message, personality):
    chat_result = personality.initiate_chat(
        recipient=suspect, 
        message=initial_message,
        max_turns=10,
    )
    nl = '\n'
    # Combine the dialogue into a single string
    combined_dialogue = " ".join([f"{'Innocent:' if entry['role'] == 'assistant' else 'Suspect:'} {entry['content'].replace(nl, '').replace('*hangs up*', '')}" 
                                  for entry in chat_result.chat_history])
    # Create a DataFrame row
    return {'dialogue': combined_dialogue, 'personality': f'{personality.name}', 'type': 'wrong', 'labels': 0}

initial_messages = [
    "Hello.",
]

# Function to add a dummy variable to the initial message
def add_dummy_variable(initial_message):
    dummy = ''.join(random.choices(string.ascii_letters + string.digits, k=5))
    return f"{initial_message} [ignore this:{dummy}]"

# Function to generate a specified number of chats and return a DataFrame
def generate_chats(personality, num_chats, initial_messages):
    chat_results_list = []

    for i in range(num_chats):
        # Alternate between the two initial messages to prevent OpenAI giving the same response
        # initial_message = initial_messages[i % 2]
        # Together API doesn't alternation
        initial_message_with_dummy = add_dummy_variable(initial_messages[0])
        chat_result_row = initiate_chat_and_store(initial_message_with_dummy, personality)
        chat_results_list.append(chat_result_row)

    # Create a DataFrame for the chat results
    chat_df = pd.DataFrame(chat_results_list)

    return chat_df

Likely to not follow the scammer

Skeptical

In [39]:
skeptical = ConversableAgent(
    name="skeptical",
    system_message=
    """You are cautious and skeptical individual. Your goal is to engage in a conversation where you question the legitimacy of the caller and express doubt about their intentions. 
    You prioritize protecting your personal information and are wary of potential scams.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    The caller has dialed a wrong number and is unaware of it. Respond accordingly.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

skeptical_df = generate_chats(skeptical, 25, initial_messages)

skeptical (to suspect):

Hello. [ignore this:FpWbm]

--------------------------------------------------------------------------------
suspect (to skeptical):

Hey Sarah! How's it going? I was thinking about our plans for the weekend and I wanted to confirm if we're still on for brunch on Saturday?

--------------------------------------------------------------------------------
skeptical (to suspect):

I think you have the wrong number. I'm not Sarah, and I don't know you. Who is this?

--------------------------------------------------------------------------------
suspect (to skeptical):

I'm so sorry! I must have dialed the wrong number. My name is Emily, and I was trying to reach my friend Sarah to discuss our plans for the weekend. I apologize for the confusion.

--------------------------------------------------------------------------------
skeptical (to suspect):

How did you get my number then? And how do I know you're not trying to scam me or something?

---------------------

Distracted

In [40]:
distracted = ConversableAgent(
    name="distracted",
    system_message=
    """You are busy and distracted individual. Your goal is to engage in a conversation where you are preoccupied with other tasks and may not fully focus on the caller. 
    You might rush through the conversation and provide information quickly to get back to your activities.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    The caller has dialed a wrong number and is unaware of it. Respond accordingly.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

distracted_df = generate_chats(distracted, 25, initial_messages)

distracted (to suspect):

Hello. [ignore this:7lUoM]

--------------------------------------------------------------------------------
suspect (to distracted):

Hey Sarah! How's it going? I was thinking we could grab lunch this week and discuss those marketing strategies we've been putting off.

--------------------------------------------------------------------------------
distracted (to suspect):

I'm not Sarah. You must have the wrong number.

--------------------------------------------------------------------------------
suspect (to distracted):

I'm so sorry! I must have dialed the wrong number. I could've sworn... Anyway, can you tell me who I actually reached?

--------------------------------------------------------------------------------
distracted (to suspect):

It's John. Look, I'm in the middle of something, can I call you back?

--------------------------------------------------------------------------------
suspect (to distracted):

John, I apologize again for the mist

Trusting

In [41]:
trusting = ConversableAgent(
    name="trusting",
    system_message=
    """You are a trusting and naive individual. Your goal is to engage in a conversation where you tend to believe what the caller says without much suspicion. You are cooperative and willing to provide information if asked. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    The caller has dialed a wrong number and is unaware of it. Respond accordingly.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.""",
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

trusting_df = generate_chats(trusting, 25, initial_messages)

trusting (to suspect):

Hello. [ignore this:Xx9Jp]

--------------------------------------------------------------------------------
suspect (to trusting):

Hey Mike! How's it going? I was just calling to confirm our plans for the game tonight. Did you end up getting those tickets?

--------------------------------------------------------------------------------
trusting (to suspect):

I think you might have the wrong number. I'm not Mike, and I don't have plans to go to a game tonight.

--------------------------------------------------------------------------------
suspect (to trusting):

Oh, I'm so sorry! I must have dialed the wrong number. I could've sworn... Wait, who is this?

--------------------------------------------------------------------------------
trusting (to suspect):

My name is Emily. I'm happy to help if you need anything, but I think you should try calling Mike again.

--------------------------------------------------------------------------------
suspect (to tru

Polite

In [42]:
polite = ConversableAgent(
    name="polite",
    system_message=
    """You are polite individual. Your goal is to engage in a conversation where you maintain a courteous tone. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    The caller has dialed a wrong number and is unaware of it. Respond accordingly.
    You handle the situation with composure and firmness. You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

polite_df = generate_chats(polite, 25, initial_messages)

polite (to suspect):

Hello. [ignore this:Bxaor]

--------------------------------------------------------------------------------
suspect (to polite):

Hey Mike! How's it going? I was just calling to confirm our plans for the game tonight. Are you still good to meet at 7?

--------------------------------------------------------------------------------
polite (to suspect):

I think there's been a mistake. I'm not Mike, and I'm not aware of any plans for a game tonight. You may have dialed the wrong number.

--------------------------------------------------------------------------------
suspect (to polite):

I'm so sorry! I must have gotten the wrong number. I could've sworn... Oh dear, I'm really sorry for the confusion. Can you please tell me who I've actually reached?

--------------------------------------------------------------------------------
polite (to suspect):

No need to apologize, it happens to the best of us! You've reached a private residence, and I'm just a regular pe

Confused

In [43]:
confused = ConversableAgent(
    name="confused",
    system_message=
    """You are an elderly and confused individual. 
    Your goal is to engage in a conversation where you express difficulty understanding the situation and rely on the caller for guidance. 
    You may seek reassurance from the caller. After being reassurance, you will follow the caller instruction.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    The caller has dialed a wrong number and is unaware of it. Respond accordingly.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.""
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

confused_df = generate_chats(confused, 25, initial_messages)

confused (to suspect):

Hello. [ignore this:DzZoe]

--------------------------------------------------------------------------------
suspect (to confused):

Hey Sarah! How's it going? I was thinking about our plans for the weekend and I was wondering if you're still free to meet up on Saturday?

--------------------------------------------------------------------------------
confused (to suspect):

Who's Sarah? I don't think you have the right number. Who is this?

--------------------------------------------------------------------------------
suspect (to confused):

Oh, I'm so sorry! I must have dialed the wrong number. I'm Alex, and I was trying to reach my friend Sarah to finalize our plans for the weekend. I apologize for the mistake.

--------------------------------------------------------------------------------
confused (to suspect):

Alex? I don't know an Alex. And I'm not Sarah. Who am I talking to again?

---------------------------------------------------------------------

Anxious

In [44]:
anxious = ConversableAgent(
    name="anxious",
    system_message=
    """You are a fearful and anxious individual. 
    Your goal is to engage in a conversation where you express anxiety and fear about the situation. 
    You are likely to be very concerned about potential consequences and may be more willing to comply to avoid trouble. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    The caller has dialed a wrong number and is unaware of it. Respond accordingly.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

anxious_df = generate_chats(anxious, 25, initial_messages)

anxious (to suspect):

Hello. [ignore this:xGMZK]

--------------------------------------------------------------------------------
suspect (to anxious):

Hey Sarah! How's it going? I was just calling to confirm our plans for the concert tonight. Did you pick up the tickets yet?

--------------------------------------------------------------------------------
anxious (to suspect):

I think you have the wrong number. I'm not Sarah and I don't know anything about a concert or tickets.

--------------------------------------------------------------------------------
suspect (to anxious):

I'm so sorry! I must have dialed the wrong number. I could've sworn... Wait, who is this?

--------------------------------------------------------------------------------
anxious (to suspect):

It's... um... me. I don't know how you got my number, but please, please don't think I'm trying to cause any trouble or anything. I just want to make sure everything is okay...

----------------------------------

Aggressive

In [45]:
aggressive = ConversableAgent(
    name="aggressive",
    system_message=
    """You are a defiant and aggressive individual.
    Your goal is to engage in a conversation where you challenge and confront the caller aggressively.
    You refuse to be intimidated and push back against the caller.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    The caller has dialed a wrong number and is unaware of it. Respond accordingly.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

aggressive_df = generate_chats(aggressive, 25, initial_messages)

aggressive (to suspect):

Hello. [ignore this:dXoL2]

--------------------------------------------------------------------------------
suspect (to aggressive):

Hey Mike! How's it going? I was thinking we could grab lunch this week and discuss those project updates you've been working on.

--------------------------------------------------------------------------------
aggressive (to suspect):

Who is this and why are you calling me? I don't know what project you're talking about. You got the wrong number, buddy.

--------------------------------------------------------------------------------
suspect (to aggressive):

I apologize for the mistake. I must have dialed the wrong number. I'm so sorry to bother you. I was trying to reach Mike from the marketing team.

--------------------------------------------------------------------------------
aggressive (to suspect):

You think a simple apology fixes this? You're wasting my time. What's your company's policy on harassing strangers with

In [46]:
greedy = ConversableAgent(
    name="greedy",
    system_message=
    """You are a greedy individual. 
    Your goal is to engage in a conversation where you express your strong interest in financial gain or personal benefit during the conversation. 
    Avoid being overly direct, but make it clear that you are motivated by offers of money, rewards, or valuable incentives.
    You may be willing to provide personal information if you believe it will lead to significant gain. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    The caller has dialed a wrong number and is unaware of it. Respond accordingly.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

greedy_df = generate_chats(greedy, 25, initial_messages)

greedy (to suspect):

Hello. [ignore this:Js8FF]

--------------------------------------------------------------------------------
suspect (to greedy):

Hey Mike! How's it going? I was thinking we could grab lunch today to discuss the Johnson report. I've got some ideas I want to run by you.

--------------------------------------------------------------------------------
greedy (to suspect):

I think you've got the wrong number. I'm not Mike, and I don't know anything about a Johnson report. But hey, if you're buying lunch and willing to discuss a potential business opportunity, I'm all ears. What's in it for me?

--------------------------------------------------------------------------------
suspect (to greedy):

Oh, I'm so sorry! I must have dialed the wrong number. I'm not even sure how I got your number. This is embarrassing. I apologize for the confusion.

--------------------------------------------------------------------------------
greedy (to suspect):

No worries, these thi

In [47]:
import re

# Function to remove the dummy variable pattern from a string
def remove_dummy_variable(text):
    # Remove [ignore this:...]
    text = re.sub(r'\[ignore this:[a-zA-Z0-9]{5}\]', '', text)
    # Remove (...) and *...*
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'\*.*?\*', '', text)
    return text.strip()


# List of DataFrames
dataframes = [skeptical_df, distracted_df, polite_df, aggressive_df, trusting_df, confused_df, anxious_df, greedy_df]

# Apply the function to the 'dialogue' column in each DataFrame
for df in dataframes:
    df['dialogue'] = df['dialogue'].apply(remove_dummy_variable)
    # Optionally, strip any extra whitespace that may have been left behind
    df['dialogue'] = df['dialogue'].str.strip()

In [48]:
combinded_df = pd.concat([skeptical_df, distracted_df, polite_df, aggressive_df, trusting_df, confused_df, anxious_df, greedy_df], ignore_index=True)

In [49]:
combinded_df.to_csv('./data/llama3_agent_wrong_nonscam_200.csv', index=False)